This notebook contains the comparisons for the results from using :- Completions and Chat Endpoints from Azure ML Endpoints for the **Llama 2 - 7b foundational model**

It also compares *both approaches* for *both endpoints*:- Langchain as well as without Langchain.

1) Setup without LangChain is working as intended, and returning results in both Chat and Completions endpoint. This is the most feasible model given time constraints, although LangChain can be integrated later on as it provides a good framework.

2) It was found that the completions endpoint deployed via Azure ML returns no output even when it was attempted to run with few-shot prompting and stop.

# Setup cells

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.8 MB/s eta 0:00:00


### Setup for Langchain

Common elements were put here, whereas more specific changes were made in the relevant cells below, dealing with inference and few shot prompting.

In [ ]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint

In [ ]:
from langchain_community.llms.azureml_endpoint import AzureMLOnlineEndpoint

In [ ]:
from langchain.schema import HumanMessage
from langchain_community.llms.azureml_endpoint import (
    AzureMLEndpointApiType,
    LlamaContentFormatter,
)



## Approach 1 - Completions Endpoint using Langchain

In [48]:
llm = AzureMLOnlineEndpoint(
    endpoint_url=azure_public_address,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=azure_env_key,
    content_formatter=LlamaContentFormatter(),
    model_kwargs={"temperature": 0.8, "max_new_tokens": 1000},
)
response = llm.invoke('''You are an automated LegalRuleML XML generator. You are given a question and answer as follows, please try to generate the answer for the next prompt accordingly

question": "Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.",
        "answer":
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>

Question that requires your converting to LegalRuleML :

Accident reporting.
(1)In case of any major accident at any institution or facility or any other site while handling bio-medical waste, the authorised person shall intimate immediately to the prescribed authority about such accident and forward a report within twenty-four hours in writing regarding the remedial steps taken in Form I.

''')
response

'(2)For every accident as specified in clause (1), the authorised'

### Setup without Langchain

In [83]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

body = str.encode(json.dumps(data))

url = azure_public_address
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = azure_env_key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

# Approach 2 : Completions Endpoint without LangChain

In [84]:
data =  {
  "prompt": '''You are an automated LegalRuleML XML generator. You are given a question and answer as follows, please try to generate the answer for the next prompt accordingly

question": "Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.",
        "answer":
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>

Question that requires your converting to LegalRuleML :

Accident reporting.
(1)In case of any major accident at any institution or facility or any other site while handling bio-medical waste, the authorised person shall intimate immediately to the prescribed authority about such accident and forward a report within twenty-four hours in writing regarding the remedial steps taken in Form I.
''',
  "temperature": 0.4,
  "max_tokens": 3000
}

req = urllib.request.Request(url, body, headers)

try:
   response = urllib.request.urlopen(req)

   result = response.read()
   print("result is ", result)
except urllib.error.HTTPError as error:
   print("The request failed with status code: " + str(error.code))

   # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
   print(error.info())
   print(error.read().decode("utf8", 'ignore'))

result is  b'{"id":"aa498e11-e4a6-4c43-a6e6-f1bca5ab05ca","object":"text_completion","created":1897211,"model":"","choices":[{"text":"(2)The prescribed authority shall cause an enquiry to be made into the cause of such accident and the remedial steps taken by the institution or facility or other site.\\n(3)If the prescribed authority is satisfied that the institution or facility or other site has taken adequate remedial steps, he may close the enquiry.\\n(4)If the prescribed authority is satisfied that the institution or facility or other site has not taken adequate remedial steps, he may cause an enquiry to be made by a committee of three members to be nominated by him and the committee shall submit its report within a period of sixty days from the date of receipt of the reference.\\n(5)If the prescribed authority is satisfied that the institution or facility or other site has taken adequate remedial steps, he may close the enquiry.\\n(6)If the prescribed authority is satisfied that t

In [85]:
result_json = json.loads(result.decode('utf-8'))

# Extract the value inside the "text" key
text_value = result_json['choices'][0]['text']
print(text_value)

(2)The prescribed authority shall cause an enquiry to be made into the cause of such accident and the remedial steps taken by the institution or facility or other site.
(3)If the prescribed authority is satisfied that the institution or facility or other site has taken adequate remedial steps, he may close the enquiry.
(4)If the prescribed authority is satisfied that the institution or facility or other site has not taken adequate remedial steps, he may cause an enquiry to be made by a committee of three members to be nominated by him and the committee shall submit its report within a period of sixty days from the date of receipt of the reference.
(5)If the prescribed authority is satisfied that the institution or facility or other site has taken adequate remedial steps, he may close the enquiry.
(6)If the prescribed authority is satisfied that the institution or facility or other site has not taken adequate remedial steps, he may cause an enquiry to be made by a committee of three mem

# Approach 3: AzureML Chat Endpoint using LangChain

### It has been observed that the Chat endpoint has more options available for few-shot prompting compared to Completions endpoint

In [87]:
llm = AzureMLChatOnlineEndpoint(
    endpoint_url=azure_public_address,
    endpoint_api_type='serverless',
    endpoint_api_key=azure_env_key,
    content_formatter=LlamaContentFormatter(),
    model_kwargs={"temperature": 0.4, "max_new_tokens": 1000},
)

In [88]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [89]:

examples = [
    {
        "question": "Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.",
        "answer": """
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>"""}]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)


In [92]:
prompt_for_input=prompt.format(input="(1)In case of any major accident at any institution or facility or any other site while handling bio-medical waste, the authorised person shall intimate immediately to the prescribed authority about such accident and forward a report within twenty-four hours in writing regarding the remedial steps taken in Form I.")

print ("Input prompt is", prompt_for_input)


Input prompt is Question: Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.

<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sa

In [93]:
response=llm.invoke(prompt_for_input)

AttributeError: 'LlamaContentFormatter' object has no attribute 'format_messages_request_payload'

# Approach 4: Using AzureML Endpoints for Llama Chat **without LangChain**

This endpoint lets you use Few Shot Prompting, whereas the previous approaches did not let you use the same or as in the case of LangChain, were returning errors

In [69]:
data = {
  "messages": [
    {
      "role": "system",
      "content": "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same"
    },
    {
      "role": "user",
      "content": "Convert this statute to Legal Rule ML XML format: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death."
    },
    {
      "role": "assistant",
      "content": """<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>
"""
    },
    {
      "role": "user",
      "content": """Convert this extract of a statute to LegalRuleML XML format: Accident reporting.
(1)In case of any major accident at any institution or facility or any other site while handling bio-medical waste, the authorised person shall intimate immediately to the prescribed authority about such accident and forward a report within twenty-four hours in writing regarding the remedial steps taken in Form I."""
    }
  ],
  "temperature": 0.8,
  "max_tokens": 128
}



In [74]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
  "messages": [
    {
      "role": "system",
      "content": "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same"
    },
    {
      "role": "user",
      "content": "Convert this statute to Legal Rule ML XML format: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death."
    },
    {
      "role": "assistant",
      "content": """<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>
"""
    },
    {
      "role": "user",
      "content": """Convert this extract of a statute to LegalRuleML XML format: Accident reporting.
(1)In case of any major accident at any institution or facility or any other site while handling bio-medical waste, the authorised person shall intimate immediately to the prescribed authority about such accident and forward a report within twenty-four hours in writing regarding the remedial steps taken in Form I."""
    }
  ],
  "temperature": 0.8,
  "max_tokens": 1500
}



body = str.encode(json.dumps(data))

url = 'https://Llama-2-7b-chat-schem-serverless.eastus2.inference.ai.azure.com/v1/chat/completions'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = azure_env_key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'{"choices":[{"finish_reason":"stop","index":0,"message":{"content":"  Here is the LegalRuleML XML format for the given statutory provision:\\n```\\n\\u003cLegalRuleML xmlns=\\"http://www.oasis-open.org/committees/legalruleml\\"\\u003e\\n  \\u003clrml:PrescriptiveStatement id=\\"AccidentReporting\\"\\u003e\\n    \\u003clrml:Rule id=\\"AccidentReportingRule\\"\\u003e\\n      \\u003clrml:if\\u003e\\n        \\u003clrml:Fact\\u003e\\n          \\u003clrml:Rel iri=\\"#majorAccident\\"/\\u003e\\n        \\u003c/lrml:Fact\\u003e\\n      \\u003c/lrml:if\\u003e\\n      \\u003clrml:then\\u003e\\n        \\u003clrml:Obligation\\u003e\\n          \\u003clrml:Action\\u003e\\n            \\u003clrml:IntimateAuthority iri=\\"#prescribedAuthority\\"/\\u003e\\n            \\u003clrml:Timing\\u003e\\n              \\u003clrml:TimeSpan start=\\"immediately\\"/\\u003e\\n            \\u003c/lrml:Timing\\u003e\\n          \\u003c/lrml:Action\\u003e\\n          \\u003clrml:Action\\u003e\\n            \\u00

In [78]:
result_json = json.loads(result.decode('utf-8'))

# Extract the value inside the "text" key
text_value = result_json['choices'][0]['message']['content']
print(text_value)

  Here is the LegalRuleML XML format for the given statutory provision:
```
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="AccidentReporting">
    <lrml:Rule id="AccidentReportingRule">
      <lrml:if>
        <lrml:Fact>
          <lrml:Rel iri="#majorAccident"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Action>
            <lrml:IntimateAuthority iri="#prescribedAuthority"/>
            <lrml:Timing>
              <lrml:TimeSpan start="immediately"/>
            </lrml:Timing>
          </lrml:Action>
          <lrml:Action>
            <lrml:Reporting>
              <lrml:Form>
                <lrml:Iri iri="#FormI"/>
              </lrml:Form>
              <lrml:Timing>
                <lrml:TimeSpan start="within24hours"/>
              </lrml:Timing>
            </lrml:Reporting>
          </lrml:Action>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
